Scikit-learn 라이브러리는 기계 학습을 위한 가장 인기있는 라이브러리 중 하나이다. 여기에서는 scikit-learn 라이브러리를 사용하여 Keras의 딥러닝 모델을 wrapping하는 방법을 이야기하고자 한다.

- Scikit-learn 기계 학습 라이브러리와 함께 사용할 Keras 모델을 wrapping하는 방법.
- scikit-learn에서 cross validation을 이용하여 Keras 모델을 평가하는 방법.
- scikit-learn에서 grid search을 이용하여 Keras 모델의 하이퍼 파라미터를 조정하는 방법.

# 개요

Keras는 Python에서 딥러닝 모델을 학습하기 위한 좋은 라이브러리이지만, 말 그대로 딥러닝을 위한 것이지, 전반적인 기계학습을 위한 것은 아니다. 실제로 Keras는 딥러닝 모델을 빠르고 간단하게 정의하고 구축하는 데 필요한 것에만 초점을 두는 미니멀리즘을 추구한다.

Python의 scikit-learn 라이브러리는 효율적인 수치 계산을 위해 SciPy 스택을 기반으로 한다. 범용 기계 학습을 위한 많은 기능을 갖춘 라이브러리이며, 딥러닝 모델 개발에 유용한 많은 유틸리티를 제공한다.

Keras 라이브러리는 딥러닝 모델을 위한 편리한 래퍼를 제공하며, 여기에서는 Keras에서 분류 신경망 모델을 생성하되, 여기에 KerasClassifier 래퍼를 통해 scikit-learn 라이브러리를 사용하는 예를 살펴볼 것이다.

# k-Fold Stratified Cross Validation으로 모델 평가

Keras의 KerasClassifier클래스와 KerasRegressor클래스는 모델 빌드를 위해 호출할 함수이며, 여기에는 build_fn 인수가 있다. 이 인수를 위해 모델을 먼저 정의하고 컴파일 한 후 그 모델을 그대로 반환하는 함수를 만들어야 한다.

아래 코드에서는 간단한 다층 신경망을 만드는 함수 create model() 함수를 정의한다.

이 함수 이름을 KerasClassifier 클래스의 build_fn 인수에 전달한다. 또한 epochs=150, batch_size=10의 추가 인수를 전달한다. 이들은 자동으로 번들로 묶여서 KerasClassifier 클래스에 의해 내부적으로 호출되는 fit() 함수에 전달된다.

아래 코드에서는 scikit-learn 라이브러리의 StratifiedKFold를 사용하여 10-fold stratified cross validation을 수행한다.

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy

# 모델을 생성하는 함수. 이 함수 자체가 KerasClassifier의 인수에 대한 입력이 될 것이다.
def create_model():
    # 모델 정의
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # 모형 적합
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# 고정 시드 값으로 초기화
seed = 123
numpy.random.seed(seed)

# Pima Indians Diabetes 데이터 불러오기
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# 입력 변수와 출력 변수를 구분하기
X = dataset[:,0:8]
Y = dataset[:,8]

# 모델 정의
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)

# 10-fold cross validaion을 이용한 모형 평가
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

Using TensorFlow backend.


0.696514024761


예제를 실행하면 10-fold cross validaion을 통해, 총 10개의 모델이 생성되고 각각 평가되며, 최종작으로는 이들을 평균한 정확도가 표시된다.

Keras 모델이 랩핑되면 이와 같이 모델을 평가하는 과정이 크게 간소화 될 수 있음을 알 수 있다.

# 딥러닝 모델의 매개변수를 Grid Search를 이용하여 탐색해보기

위의 코드는 Keras의 딥러닝 모델을 래핑하고 scikit-learn 라이브러리의 함수에서 사용하는 것이 얼마나 쉬운지를 보여주기 위한 코드였다.

이제 한 단계 더 나아가보자. 위 예제로부터 우리는 fit() 함수에 인수를 전달할 수 있다는 것을 이미 알고 있다. KerasClassifier 래퍼를 만들 때 build_fn 인수에 지정하기 위한 함수에도 인수를 사용할 수 있다.

또한 다음 코드에서는 grid search를 이용하여 신경망 모델의 여러 구성을 평가하고 최상의 성능을 제공하는 조합을 찾아보고자 한다. create_model() 함수에는 optimizer와 init이라는 인수가 있는데, 둘 다 기본 값이 있어야 한다. 이를 통해 네트워크에 대한 다양한 최적화 알고리즘과 가중치 초기화 체계를 사용했을 때의 효과를 평가할 수 있다.

우선 모델을 만든 후에, 검색하고자 하는 매개변수의 값 배열을 정의한다. 구체적으로는 다음과 같습니다.

- Optimizer
- Initializer
- Number of epochs
- Batches

옵션이 사전에 지정되고, GridSearchCV라는 scikit-learn 클래스에 내장된 함수로 전달된다. 이 클래스는 optimizer, initializer, epoch 및 batch의 조합에 대한 매개 변수(2 × 3 × 3 × 3)의 각 조합에 따른 각 신경망 모델을 평가한다. 그런 다음 각 조합은 기본값으로 3-fold stratified cross validation을 통해 평가가 된다.

사실 이러한 과정은 계산에 걸리는 시간 때문에 가볍게 사용하기 위한 방법은 아니다. 합리적인 시간 내에 완료 할 수 있는 더 작은 데이터 subset을 이용하여 작은 실험설계를 하는 것이 유용 할 수 있다. 아래 코드도 CPU만 사용하는 워크스테이션에는 구동이 완료되기까지 대략 5분이 넘는 시간이 걸릴 수 있다.

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy

# 모델을 생성하는 함수. 이 함수 자체가 KerasClassifier의 인수에 대한 입력이 될 것이다.
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # 모델 정의
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # 모형 적합
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# 고정 시드 값으로 초기화
seed = 123
numpy.random.seed(seed)

# Pima Indians Diabetes 데이터 불러오기
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# 입력 변수와 출력 변수를 구분하기
X = dataset[:,0:8]
Y = dataset[:,8]

# 모델 정의
model = KerasClassifier(build_fn=create_model, verbose=0)

# epochs, batch size, optimizer의 Grid Search
optimizers = ['rmsprop', 'adam']
inits = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)

# 결과 요약
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.760417 using {'batch_size': 10, 'epochs': 150, 'optimizer': 'adam', 'init': 'uniform'}
0.690104 (0.025976) with: {'batch_size': 5, 'epochs': 50, 'optimizer': 'rmsprop', 'init': 'glorot_uniform'}
0.658854 (0.043537) with: {'batch_size': 5, 'epochs': 50, 'optimizer': 'adam', 'init': 'glorot_uniform'}
0.692708 (0.009207) with: {'batch_size': 5, 'epochs': 50, 'optimizer': 'rmsprop', 'init': 'normal'}
0.716146 (0.025780) with: {'batch_size': 5, 'epochs': 50, 'optimizer': 'adam', 'init': 'normal'}
0.694010 (0.041626) with: {'batch_size': 5, 'epochs': 50, 'optimizer': 'rmsprop', 'init': 'uniform'}
0.723958 (0.018688) with: {'batch_size': 5, 'epochs': 50, 'optimizer': 'adam', 'init': 'uniform'}
0.731771 (0.020505) with: {'batch_size': 5, 'epochs': 100, 'optimizer': 'rmsprop', 'init': 'glorot_uniform'}
0.734375 (0.012758) with: {'batch_size': 5, 'epochs': 100, 'optimizer': 'adam', 'init': 'glorot_uniform'}
0.740885 (0.022402) with: {'batch_size': 5, 'epochs': 100, 'optimizer': 'rmsprop'